In [ ]:
import json

# Read the JSON file
with open('C:/Users/ghaza/Downloads/integrated_data.json') as file:
    data = json.load(file)

# Extract ingredient names and categories
ingredients = []
for ingredient in data:
    ingredient_name = ingredient['ingredients']
    ingredient_category = ingredient['category']
    ingredients.append({'name': ingredient_name, 'category': ingredient_category})


# Save the ingredients to a JSON file
output_file = 'C:/Users/ghaza/Desktop/ingredients.json'
with open(output_file, 'w') as file:
    json.dump(ingredients, file, indent=4)





        

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
import pickle

def calculate_and_save_euclidean_distances(embedding_file, output_file):
    # Load ingredient embeddings from file
    with open(embedding_file, "rb") as file:
        ingredient_embeddings = pickle.load(file)

    # Calculate Euclidean distance matrix
    ingredient_embeddings_list = list(ingredient_embeddings.values())
    euclidean_distance_matrix = euclidean_distances(ingredient_embeddings_list)

    # Get the list of ingredient names
    ingredient_names = list(ingredient_embeddings.keys())

    # Save the Euclidean distance matrix to a file
    with open(output_file, "w", encoding="utf-8") as file:
        for i, ingredient1 in enumerate(ingredient_names):
            for j, ingredient2 in enumerate(ingredient_names):
                distance = euclidean_distance_matrix[i, j]
                line = f"{ingredient1}, {ingredient2}: {distance}\n"
                file.write(line)


In [ ]:
import numpy as np

def load_distance_file(distance_file):
    ingredient_distances = {}
    with open(distance_file, "r", encoding="utf-8") as file:
        for line in file:
            ingredients, distance = line.strip().split(":")
            ingredient1, ingredient2 = ingredients.split(",")
            distance = float(distance)
            
            if ingredient1 not in ingredient_distances:
                ingredient_distances[ingredient1] = {}
            if ingredient2 not in ingredient_distances:
                ingredient_distances[ingredient2] = {}
            
            ingredient_distances[ingredient1][ingredient2] = distance
            ingredient_distances[ingredient2][ingredient1] = distance
    
    return ingredient_distances

import json






    

In [ ]:
def find_replacements(ingredient, ingredient_distances, ingredient_category_data):
    # Load ingredient categories from JSON data
    ingredient_categories = {}
    for ingredient_data in ingredient_category_data:
        ingredient_name = ingredient_data['name']
        categories = ingredient_data['category']
        ingredient_categories[ingredient_name] = categories

    # Get the distances for the input ingredient
    distances = ingredient_distances.get(ingredient, {})

    # Sort the distances in ascending order
    sorted_distances = sorted(distances.items(), key=lambda x: x[1])

    # Find the best 20 ingredient replacements with distances
    replacements = []
    for ingredient, distance in sorted_distances:
        ingredient_categories_list = ingredient_categories.get(ingredient.strip(), [])

        if ingredient_categories_list and all(category not in ingredient_categories_list for category in ['Meat', 'Seafood', 'Fish', 'Fungus']):
            replacements.append((ingredient, distance))

        # Break the loop when we have found 20 replacements
        if len(replacements) == 13:
            break

    return replacements






In [ ]:
import json

ingredient = "Egg"
ingredient_distances = load_distance_file("C:/Users/ghaza/Desktop/embeddings-1&2/distance1.txt")

# Load ingredient category data from JSON
with open("C:/Users/ghaza/Desktop/ingredients.json", "r") as file:
    ingredient_category_data = json.load(file)

replacements = find_replacements(ingredient, ingredient_distances, ingredient_category_data)
print("Replacements:")
for replacement, distance in replacements:
    if replacement != ingredient and distance != 0.0:  # Exclude the input ingredient and distance 0.0
        print("-", replacement, "(Distance:", distance, ")")
